In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
# %% load packages
import locale
import sys
import os
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime


c:\Users\admin\anaconda3\envs\DLiE\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, SimpleMLP, train_mlp, \
  prepare_train_test_tensors, build_mlp_rolling_forecasts, tune_mlp_hyperparameters, \
  DST_trafo

from srs.utils.our_utils import run_forecast_step
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset



In [4]:
# Transform merged dataset using DST_trafo and prepare training data.

repo_root = Path.cwd().parents[1]
data_no1 = pd.read_csv(repo_root / "data" /'data_no1.csv')
data_t_no1, train_t_no1, train_dates, price_t_no1 = prepare_dataset_tensor(
    repo_root / "data" / "data_no1.csv",
    tz="CET",
    seed=42,
    test_days=2*365,         
    dtype=torch.float64, 
)

data_array = data_t_no1         
price_S    = price_t_no1        
dates_S    = train_dates    

D          = 730            
S          = 24
WD         = [1, 6, 7]
PRICE_S_LAGS = [1, 2, 7]
da_lag = [0]

#validation period length
length_eval = 2 * 365

# The first obdervation in the evaluation period
begin_eval = data_array.shape[0] - length_eval

N_s = length_eval

model_names = [
    "true",
    "expert_ext",
    "linar_gam",
    "light_gbm"
]
n_models = len(model_names)

# 3D tensor to hold forecasts:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
forecasts = torch.full((N_s, S, n_models), float('nan'), dtype=torch.float64, device=device)

In [6]:
data_array.shape

torch.Size([2193, 24, 10])

In [5]:
# Create thread pool
init_time = datetime.now()
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(
            run_forecast_step,
            n,
            price_S,
            data_array,
            begin_eval,
            D,
            dates_S,
            WD,
            PRICE_S_LAGS,
            da_lag,
            data_no1.columns[1:],  # reg_names
            data_no1.columns[1:]   # data_columns
        )
        for n in range(N_s)
    ]

    for future in as_completed(futures):
        try:
            n, gam_24h, gam_per_hour = future.result()
            forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
            forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)
            #forecasts[n, :, insert_order] = true_price
            #forecasts[n, :, insert_order] = torch.tensor(expert, dtype=forecasts.dtype, device=forecasts.device)
            #forecasts[n, :, insert_order] = torch.tensor(lg_gbm, dtype=forecasts.dtype, device=forecasts.device)
        except Exception as e:
            print(f"Thread crashed: {e}")

# End timing
end_time = datetime.now()
duration_minutes = (end_time - init_time).total_seconds() / 60
print(f"\nParallel training duration (threaded): {duration_minutes:.2f} minutes")

START NS: 0  index: 1463  data shape: torch.Size([2193, 24, 10])
START NS: 1  index: 1464  data shape: torch.Size([2193, 24, 10])
START NS: 2  index: 1465  data shape: torch.Size([2193, 24, 10])
START NS: 3  index: 1466  data shape: torch.Size([2193, 24, 10])
START NS: 4  index: 1467  data shape: torch.Size([2193, 24, 10])
START NS: 5  index: 1468  data shape: torch.Size([2193, 24, 10])
START NS: 6  index: 1469  data shape: torch.Size([2193, 24, 10])
START NS: 7  index: 1470  data shape: torch.Size([2193, 24, 10])
shape of initial data: torch.Size([731, 24, 10])
missing values of initial data: tensor([ 0,  0,  0,  0,  0, 48,  0,  0,  0,  0])
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1])
shape of initial data: torch.Size([731, 24, 10])
missing values of initial data: tensor([ 0,  0,  0,  0,  0, 48,  0,  0,  0,  0])
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 2, 0, 0,

C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)


torch.Size([731, 15])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2])
GAM fitted for hour 23
END NS: 1
START NS: 9  index: 1472  data shape: torch.Size([2193, 24, 10])


C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)


GAM fitted for hour 23
END NS: 0
START NS: 10  index: 1473  data shape: torch.Size([2193, 24, 10])
GAM fitted for hour 23


...

C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)


GAM fitted for hour 6
torch.Size([731, 16])
Missing values per column before we remove: tensor([1, 1, 2, 7, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 1])
GAM fitted for hour 8
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1])
GAM fitted for hour 7
GAM fitted for hour 9
torch.Size([731, 16])
Missing values per column before we remove: tensor([1, 1, 2, 7, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 1])
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1])
GAM fitted for hour 8
GAM fitted for hour 10
torch.Size([731, 16])
Missing values per column before we remove: tensor([1, 1, 2, 7, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 1])
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1])
GAM fitted for hour 11
GAM fitted for hour 9
torch.Size([731, 16])
Missing values per column before we remove: tensor([0, 1, 2, 7

C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)


GAM fitted for hour 22
torch.Size([731, 15])
Missing values per column before we remove: tensor([1, 1, 2, 7, 1, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2])
GAM fitted for hour 23
END NS: 729

Parallel training duration (threaded): 108.12 minutes


C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 0] = torch.tensor(gam_24h, dtype=forecasts.dtype, device=forecasts.device)
C:\Users\admin\AppData\Local\Temp\ipykernel_11152\1115178007.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  forecasts[n, :, 1] = torch.tensor(gam_per_hour, dtype=forecasts.dtype, device=forecasts.device)


In [24]:
# save forecasts (v1 = version 1, add up as number of experiments increases)
fc = forecasts.cpu().numpy()
N_s, S, n_models = fc.shape

samples = np.repeat(np.arange(N_s), S)
hours = np.tile(np.arange(S), N_s)
data = {
    "sample": samples,
    "hours": hours
}

for name, m in [("gam_24h", 0), ("gam_1h", 1)]:
    data[name] = fc[:,:,m].reshape(-1)
    
df = pd.DataFrame(data)
df.to_csv(repo_root/"data"/"forecasts_gam24h_gam_1h_v1.csv", index=False)
print(f"Saved forecasts with columns: {df.columns.tolist()}")

Saved forecasts with columns: ['sample', 'hours', 'gam_24h', 'gam_1h']


In [ ]:
# print(data_array.shape )
# print(price_S.shape )
# print(dates_S.shape )

# print(data_t_no1.shape)
# print(price_t_no1.shape)
# print(train_dates.shape)
# print(train_t_no1.shape)

torch.Size([2193, 24, 10])
torch.Size([2193, 24])
(2193,)
